Here's a simplified example of how Deepseek might be fine-tuned for a classification task without any sensitive data. Before we begin, let's install the necessary Python packages to ensure the code runs smoothly. These include transformers for model handling and torch for tensor processing.

### Install the required libraries: Run the following command to install the required libraries:

In [1]:
! pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

#### Now, you can import the necessary libraries for using the Deepseek model:

In [ ]:
import torch

from torch.utils.data import Dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

Loading pre-trained Deepseek model and tokenizer: We load the pre-trained Deepseek tokenizer and model to use for sentiment classification:

AutoTokenizer.from_pretrained: Loads the pre-trained Deepseek tokenizer

AutoModelForSequenceClassification.from_pretrained: Loads the pre-trained Deepseek model for sequence classification tasks

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-base", trust_remote_code=True)

model = AutoModelForSequenceClassification.from_pretrained(

   "deepseek-ai/deepseek-llm-7b-base",

   num_labels=2,  # Binary classification

   trust_remote_code=True

)



# Set padding token if not already set

if tokenizer.pad_token is None:

   tokenizer.pad_token = tokenizer.eos_token

   model.config.pad_token_id = tokenizer.pad_token_id

Example text and data tokenization: Here we define two example text samples and their corresponding sentiment labels, and tokenize the input text to convert it into a format suitable for Deepseek.

tokenizer: Converts the text into token IDs that Deepseek can process

padding=True: Ensures the text is padded to the same length

truncation=True: Truncates longer texts to fit Deepseek's maximum input length

return_tensors="pt": Returns PyTorch tensors for compatibility with the model

In [ ]:
texts = ["I love machine learning.", "This is an amazing tutorial on Deeplearning."]

labels = [1, 0]  # Labels for sentiment (1 = positive, 0 = negative)

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")



# Custom Dataset class

class CustomDataset(Dataset):

   def __init__(self, inputs, labels):

       self.input_ids = inputs['input_ids']

       self.attention_mask = inputs['attention_mask']

       self.labels = torch.tensor(labels)



   def __len__(self):

       return len(self.labels)



   def __getitem__(self, idx):

       return {

           'input_ids': self.input_ids[idx],

           'attention_mask': self.attention_mask[idx],

           'labels': self.labels[idx]

       }

#### Prepare the dataset for training: We create a dataset from the tokenized inputs and the labels:

In [ ]:
train_data = torch.utils.data.TensorDataset(inputs['input_ids'], torch.tensor(labels))

Next, define training arguments. We specify the settings for the training process, such as the number of epochs, batch size, and logging:

In [ ]:
training_args = TrainingArguments(

   output_dir="./results",

   num_train_epochs=3,

   per_device_train_batch_size=1,  # Reduced batch size due to model size

   logging_dir="./logs",

   logging_steps=10,

   save_steps=50,

   learning_rate=1e-5,

   gradient_accumulation_steps=16,  # Add gradient accumulation for large model

   fp16=True,  # Enable mixed precision training

   gradient_checkpointing=True  # Enable gradient checkpointing to save memory

)

#### Next, we fine-tune the Deepseek model with the prepared training data using the Trainer API:

In [ ]:
trainer = Trainer(

   model=model,

   args=training_args,

   train_dataset=train_data

)

trainer.train()